In [1]:
using Distributed

In [2]:
addprocs(16)

16-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17

In [3]:
@everywhere begin
    import Pkg
    Pkg.activate(joinpath(@__DIR__, "SlurmCLI"))
    Pkg.instantiate()
end

      From worker 5:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 16:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 10:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 8:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 3:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 6:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 13:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 15:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 17:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 7:	  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`
      From worker 14:	  Activating project at `/global/u1/b/blaschke/analyze-iris/Slu

  Activating project at `/global/u1/b/blaschke/analyze-iris/SlurmCLI`


In [4]:
@everywhere using SlurmCLI

In [5]:
@everywhere begin
    using Base: @kwdef
    using CodecZlib
    using DelimitedFiles
    using Chain
    using JSON
    using Dates
end

In [6]:
@everywhere @kwdef struct Job
    id::Int64                    = 0
    admin_comment::Vector{UInt8} = UInt8[]
    valid::Bool                  = false
end

In [7]:
fn_ret_type(fn, in_type::DataType) = Base.return_types(fn, (in_type,))[1]

function launch_monitor(processor; buffer_size=32)
    function remote_monitor(fn, jobs, results)
        @sync while true
            job = try
                take!(jobs)
            catch y
                break
            end
            t = @async fn(job)
            @async put!(results, fetch(t))
        end
    end

    jobs    = RemoteChannel(()->Channel{Job}(buffer_size))
    results = RemoteChannel(
            ()->Channel{fn_ret_type(processor, Job)}(buffer_size)
        )

    for p in workers()
        remote_do(remote_monitor, p, processor, jobs, results)
    end

    return jobs, results
end


function launch_consumer(processor, jobs; buffer_size=32)
    function remote_monitor(fn, jobs, results)
        @sync while true
            job = try
                take!(jobs)
            catch y
                break
            end
            t = @async fn(job)
            @async put!(results, fetch(t))
        end
    end

    # jobs    = RemoteChannel(()->Channel{Job}(buffer_size))
    results = RemoteChannel(
            ()->Channel{fn_ret_type(processor, Job)}(buffer_size)
        )

    for p in workers()
        remote_do(remote_monitor, p, processor, jobs, results)
    end

    return results
end


function collect!(
        results::A; collect_time=1,
    ) where {
             T <: Job,
             S <: AbstractChannel{T},
             A <: Union{S, RemoteChannel{S}}
            }

    collected = Vector{Job}()

    t = @async while true
        fd = take!(results)
        push!(collected, fd)
    end

    sleep(collect_time)
    schedule(t, InterruptException(), error=true)

    return collected
end

collect! (generic function with 1 method)

In [8]:
jobs, admin_comments = launch_monitor(
    x->begin
        sacct = SlurmCLI.shell(`sacct -nPXo AdminComment -j $(x.id)`)
        valid = length(sacct.stdout)>0 && length(sacct.stderr)==0
        sacct_compressed = transcode(ZlibCompressor, sacct.stdout)
        Job(id=x.id, admin_comment=sacct_compressed, valid=valid)
    end;
    buffer_size=1024
)

(RemoteChannel{Channel{Job}}(1, 1, 210), RemoteChannel{Channel{Job}}(1, 1, 211))

In [9]:
results = launch_consumer(
    x->begin
        if ! x.valid
            return x
        end

        decompressed_comment = @chain x.admin_comment begin
            transcode(ZlibDecompressor, _)
            String
        end
        path = joinpath(
            ENV["CFS"], "nstaff", "blaschke", "slurm", "cori_$(myid()).stream"
        )
        open(ZlibCompressorStream, path, "a") do stream
            size::Int64 = length(decompressed_comment)
            write(stream, size)
            write(stream, decompressed_comment)
        end
        
        return x
    end,
    admin_comments;
    buffer_size=1024
)

RemoteChannel{Channel{Job}}(1, 1, 212)

In [10]:
@async for i=1:100000000
    put!(jobs, Job(id=i))
end

Task (runnable) @0x0000155505f4ef80

In [11]:
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear)

false

In [ ]:
p  = Progress(
    3500; desc="Collected: ", showspeed=true, enabled=true
)
update!(p, 0)

while true
    admn = collect!(admin_comments)
    next!(p)

    if length(admn) == 0
        sleep(10)
        continue
    end
end

Collected: 100%|██████████████████████████| Time: 0:58:30 ( 1.00  s/it)
Worker 11 terminated.
Unhandled Task ERROR: IOError: read: connection reset by peer (ECONNRESET)
Stacktrace:
  [1] wait_readnb(x::Sockets.TCPSocket, nb::Int64)
    @ Base ./stream.jl:410
  [2] (::Base.var"#wait_locked#674")(s::Sockets.TCPSocket, buf::IOBuffer, nb::Int64)
    @ Base ./stream.jl:944
  [3] unsafe_read(s::Sockets.TCPSocket, p::Ptr{UInt8}, nb::UInt64)
    @ Base ./stream.jl:950
  [4] unsafe_read
    @ ./io.jl:759 [inlined]
  [5] unsafe_read(s::Sockets.TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
    @ Base ./io.jl:758
  [6] read!
    @ ./io.jl:760 [inlined]
  [7] deserialize_hdr_raw
    @ /global/common/cori_cle7up03/software/julia/julia/1.8.0-beta1/share/julia/stdlib/v1.8/Distributed/src/messages.jl:167 [inlined]
  [8] message_handler_loop(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool)
    @ Distributed /global/common/cori_cle7up03/software/julia/julia/1.8.0-beta1